# Named Entity Recognition (NER) with Spacy

The purpose of this notebook is to demonstrate the entire process of name-entity recognition([**NER**](https://nlp.stanford.edu/ner/)) from start to the end with [**Spacy**](https://spacy.io/). 
This notebook also explore **pattern matching** as an alternative to **NER** when there is a known small set of fixed values. 

This will be a complete end-to-end demonstration of the entire process, including both labelling and model training.

In this notebook, we train a model to detect entities related to **oil/petrol** from this [public dataset](https://www.kaggle.com/mitusha/email-dataset) which contains a list of emails related to the oil industry. This is an over simplification because we want more generic entities, but it shows how pattern matching is a better alternative than NER in this case. To summarise, we will extract oil-related elements from email messages.

Below are the process perform in this notebook:
- Read the emails data set which has an email per line.
- Label the emails with the OIL entity using **[Doccano](http://doccano.herokuapp.com/)** labeling tool. This is a manual process.
- Save the labels in a text file as **JSONL**.
- Use **Spacy** Neural Network model to train a new statistical model. 
- Save the model.
- Create a **Spacy** NLP pipeline and use the new model to detect oil entities never seen before.
- Use pattern matching instead of a deep learning model to compare both method.



## Label the Data

First Step: Level the data using open source platform **Doccano**.

Follow **[Doccano](https://doccano.github.io/doccano/tutorial/)** instructions to install and open Doccano.

If you use **Linux/Mac**, I recommend using the docker image:
- `docker pull doccano/doccano`
- `docker container create --name doccano -e "ADMIN_USERNAME=admin" -e "ADMIN_EMAIL=admin@example.com" -e "ADMIN_PASSWORD=password" -p 8000:8000 doccano/doccano`
- `docker container start doccano`
    
For **Windows**, just use **pip**: 
- `pip install doccano`
- `doccano`

Go to http://127.0.0.1:8000/.

Next, label the data using Doccano. Find entities which talk about oil, petrol, petroleum, etc and label them with the tag **OIL**. 

Export the result as **JSONL(Text label)** format.

## Model Training

First, let's read the JSONL file using format:

`{"id": 15, "text": "....", "meta": {}, "annotation_approver": null, "labels": [[226, 234, "OIL"]]}
`

In [1]:
import json
labeled_data = []
with open(r"emails_labeled.jsonl", "r") as read_file:
    for line in read_file:
        data = json.loads(line)
        labeled_data.append(data)

#### Convert the format to spacy format

Next, let's convert the Deccano format to Spacy format.

We will also remove extra columns and rename labels to entities.

In [2]:
TRAINING_DATA = []
for entry in labeled_data:
    entities = []
    for e in entry['labels']:
        entities.append((e[0], e[1],e[2]))
    spacy_entry = (entry['text'], {"entities": entities})
    TRAINING_DATA.append(spacy_entry)

### Train the model!

To avoid overfitting, we use Deep Learning (NN) with a 0.3 dropout rate.

The concept is to employ a Neural Network with multiple layers and many neurons. We show them text that has previously been classified, so we already know the answer. We'll run numerous iterations, and on each one, we'll calculate the error using a Loss Function, which will cause the weight of the neurons to be adjusted, causing them to activate. As time goes on, the network's weight will be adjusted to reduce incorrect learning patterns and solve the problem.

We randomly delete certain neurons on each iteration to avoid overfitting, which implies the model "memorises" the training data and does not perform well with fresh data. This allows the model to generalise better.

Remember to install Spacy first:
```
pip install -U pip setuptools wheel
pip install -U spacy
python -m spacy download en_core_web_sm
```

In [3]:
import spacy
import random
import json

nlp = spacy.blank("en")
ner = nlp.create_pipe("ner")
nlp.add_pipe(ner)
ner.add_label("OIL")

# Start the training
nlp.begin_training()

# Loop for 40 iterations
for itn in range(40):
    # Shuffle the training data
    random.shuffle(TRAINING_DATA)
    losses = {}

    # Batch the examples and iterate over them
    for batch in spacy.util.minibatch(TRAINING_DATA, size=2):
        texts = [text for text, entities in batch]
        annotations = [entities for text, entities in batch]

        # Update the model
        nlp.update(texts, annotations, losses=losses, drop=0.3)
    print(losses)

/Users/risehill/opt/anaconda3/lib/python3.8/site-packages/spacy/language.py:635: UserWarning: [W033] Training a new parser or NER using a model with an empty lexeme normalization table. This may degrade the performance to some degree. If this is intentional or this language doesn't have a normalization table, please ignore this warning.
  proc.begin_training(
/Users/risehill/opt/anaconda3/lib/python3.8/site-packages/spacy/language.py:635: UserWarning: [W034] Please install the package spacy-lookups-data in order to include the default lexeme normalization table for the language 'en'.
  proc.begin_training(


{'ner': 1973.4865596213326}
{'ner': 113.57156751269132}
{'ner': 217.56983780162102}
{'ner': 135.77832285394533}
{'ner': 18.947545839105487}
{'ner': 2.3462759127285975}
{'ner': 2.7424490023819144}
{'ner': 0.2527100517836849}
{'ner': 1.9989370382904463}
{'ner': 2.4769654040015526e-06}
{'ner': 6.686036946592199e-05}
{'ner': 4.48429154853934e-07}
{'ner': 2.35069317865294e-06}
{'ner': 4.132060679846962e-06}
{'ner': 3.0780311702707956e-05}
{'ner': 4.0529153954865305e-06}
{'ner': 8.623072892827798e-08}
{'ner': 2.3936120363095938e-08}
{'ner': 9.26201417283012e-07}
{'ner': 6.676562402811059e-05}
{'ner': 0.8652161449037427}
{'ner': 3.6356021931840714}
{'ner': 0.6382250019502252}
{'ner': 0.7674873296283516}
{'ner': 3.059514327155964e-05}
{'ner': 0.009003881051844805}
{'ner': 0.00015068832124390008}
{'ner': 0.3304007721099634}
{'ner': 1.6587583378080304e-10}
{'ner': 1.8754305088359436e-05}
{'ner': 2.1138066257901233e-11}
{'ner': 8.570752098763121e-10}
{'ner': 2.6192045906057025e-10}
{'ner': 1.1126

You should see the error decreasing as iterations go by, note that some times it may increase due to the dropout setting.

#### Save the model to disk

In [8]:
nlp.to_disk("oil.model")

## Test the model

Let's test the model.  For this we use displacy which will display the entities in the text.

In [5]:
from spacy import displacy
example = "service postings marathon petroleum co said it reduced the contract price it will pay for all grades of service oil one dlr a barrel effective today the decrease brings marathon s posted price for both west texas intermediate and west texas sour to dlrs a bbl the south louisiana sweet grade of service was reduced to dlrs a bbl the company last changed its service postings on jan reuter"

In [6]:
doc = nlp(example)
displacy.render(doc, style='ent')

### Conclusion

We have shown how to label data with **Doccano** and create a custom model with **Spacy**. 

## Phrase Matching

The second approach is to use pattern matching to look for certain keywords and patterns in the text. 

**Spacy** provides matchers which can be easily used to look for specific substrings, digits, etc. We can also set rules based on the part-of-speech tags.

In [9]:
import spacy
# Import the Matcher
from spacy.matcher import Matcher

nlp = spacy.load(name='en_core_web_sm')
doc = nlp(example)

# Initialize the Matcher with the shared vocabulary
matcher = Matcher(nlp.vocab)

# Add the pattern to the matcher
matcher.add("OIL_PATTERN", None, [{"LOWER": "oil"}], [{"LOWER": "petroleum"}])

# Use the matcher on the doc
matches = matcher(doc)
print("Matches:", [doc[start:end].text for match_id, start, end in matches])

Matches: ['petroleum', 'oil']


We can see how we also found the right tag, but in this case typos or similar words are not detected.

## Conclusion

We have seen to different approaches used for entity recognition. 

**Pattern Matching** can be used in the following use cases:
- Low cardinality attributes
- Common patterns such dates, quantities, numbers, etc.
- Patterns occuring in certain parts of the speech
- When typos are not expected
- Structured data



**Statistical Models** are great to learn complex patterns in the data and can "guess" and categorize data never seem before. Use cases:
- High Cardinality attributes
- You need to deal with typos (fuzzy matches)
- You need to categorize new, never seen data.
- Unstructured data

These models are much more powerful since they can make decisions on things that were never trained on. It can detect new entities without any code change.
